In [1]:
import os
import json
import pandas as pd
import numpy as np

In [2]:
no_kg_outputs = 'outputs/no_kg_results.jsonl'
text_kg_outputs = 'outputs/text_kg_results.jsonl'

In [3]:
all_results = []

In [4]:
with open(no_kg_outputs, 'r') as f:
	no_kg_results = [json.loads(line) for line in f.readlines()]
with open(text_kg_outputs, 'r') as f:
	text_kg_results = [json.loads(line) for line in f.readlines()]
with open('/home/manish/thesis-implementations/quest_generation/llama2/data/val.jsonl', 'r') as f:
	val_set = [json.loads(line) for line in f.readlines()]

In [5]:
for no_kg_result, text_kg_result, val in zip(no_kg_results, text_kg_results, val_set):
    result = {
    	'desc' : val['quest']['description'],
		'game' : val['game'],
		'no_kg_input': no_kg_result['input'],
		'text_kg_input': text_kg_result['input'],
		'no_kg': no_kg_result['output_gen'][:-10][11:],
		'text_kg': text_kg_result['output_gen'][:-10][11:],
  		'actual': no_kg_result['output_actual'][:-10][11:],
	}
    all_results.append(result)

In [6]:
results_df = pd.DataFrame(all_results)

In [7]:
results_df.head()

,desc,game,no_kg_input,text_kg_input,no_kg,text_kg,actual
0,Thank you for rescuing me. I had recovered the...,TL2,### Plots:\nthe player freed Elder Josimon fro...,### Background:\nElder Josimon is a character....,Title: The passkey ember\nObjective: Bring eld...,Title: The elder's request\nObjective: Bring e...,Title: Up in smoke\nObjective: Put the slavers...
1,Name's Valdr. I hunt out of Falkreath.\nWe tra...,TESS,"### Plots:\nValdr, Ari and Niels tracked a bea...",### Background:\nValdr is a character. Valdr i...,Title: The spriggans\nObjective: Find the spri...,Title: Moss mother cavern\nObjective: Kill the...,Title: Moss mother cavern\nObjective: Help val...
2,My daughter is gone! I left my home in your ca...,BG2,### Plots:\nGarren Windspear left his home in ...,### Background:\nGarren Windspear is a charact...,Title: The windspear daughter\nObjective: Help...,Title: The windspear family\nObjective: Rescue...,Title: Rescue garren's child from firkraag\nOb...
3,You're really going to the Blue Palace? That p...,TESS,### Plots:\nthe player told Taarie that the pl...,### Background:\nTaarie is a character. Taarie...,Title: A gift for the queen\nObjective: Bring ...,Title: The blue palace\nObjective: Deliver a l...,Title: Fit for a jarl\nObjective: Promote the ...
4,Why are you people doing this? Is it informati...,TESS,### Plots:\nthe player is torturing their pris...,### Background:\nSolitude is a location. Solit...,Title: The prisoner's inheritance\nObjective: ...,Title: A secret to die for\nObjective: Find th...,Title: The inheritance\nObjective: Find the hi...


In [16]:
out_file = os.path.join('outputs', f'all_results.jsonl')
with open(out_file, 'w') as outfile:
	for result in all_results:
		json.dump(result, outfile)
		outfile.write('\n')

In [8]:
import bert_score
from bert_score import BERTScorer
bert_score.__version__

import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [9]:
scorer = BERTScorer(model_type='microsoft/deberta-xlarge-mnli', lang="en")

In [10]:
no_kg_out = [x['no_kg'] for x in all_results]
tree_kg_out = [x['text_kg'] for x in all_results]
actual_out = [x['actual'] for x in all_results]

In [11]:
n_t_P, n_t_R, n_t_F1 = scorer.score(no_kg_out, tree_kg_out)
n_a_P, n_a_R, n_a_F1 = scorer.score(no_kg_out, actual_out)
t_a_P, t_a_R, t_a_F1 = scorer.score(tree_kg_out, actual_out)

In [12]:
results_df['noXtext'] = n_t_F1.tolist()

In [19]:
most_diff = results_df.sort_values(by=['noXtext'], ascending=True).head(20).index.tolist()
most_diff_res = [all_results[i] for i in most_diff]

In [15]:
out_file = os.path.join('outputs', f'most_diff_results.jsonl')
with open(out_file, 'w') as outfile:
	for result in most_diff_res:
		json.dump(result, outfile)
		outfile.write('\n')

In [20]:
for res in most_diff_res:
    for k,v in res.items():
        print(f'{k}:\n{v}\n--------\n')
    print('-' * 40)

desc:
Wake up! That's right, it's time to wake up, you drunken blasphemer!
Of course! You don't remember getting here. I'm guessing you also don't remember coming in here and blathering incoherently about marriage or a goat. Which means you don't remember losing your temper and throwing trash all over the temple.
You want to know where your friend Sam has gone? Dibella teaches love and compassion, but that doesn't mean we're just going to tell you what you want to know and let you walk away from this. Pick up your mess, then apologize, and if we think you're sincere we'll consider lending you aid.
--------

game:
TESS
--------

no_kg_input:
### Plots:
Senna wakes the player up: the player does not remember anything about coming to the Temple of Dibella, blathering about marriage or a goat, and throwing trash all over the Temple of Dibella

The quest related to the above information is as follows:
--------

text_kg_input:
### Background:
Senna is a character. Senna is present in Temple 